# OKM Van Loon #

This script reads in an Excel file containing a Bill of Material (BOM), and an Excel file containing prices per ingredient. It then combines these two to calculate the cost of the products. It then outputs these costs as an Excel file.

## Set-up ##

### Imports ###

In [3]:
import openpyxl
import pandas as pd

### Objects ###

In [4]:
class recipe:
    """ a recipe """

    def __init__(self, name : str, id : str, data : pd.DataFrame, cost: float=0.0, HFs: list=[]) -> None:
        """ initialise an instance of recipe"""
        self.name = name
        self.id = id
        self.data = data
        self.cost = cost
        self.HFs = HFs

    def __str__(self) -> str:
        """ set the string representation of a recipe """
        return f'{self.id} {self.name}'
    
class HF:
    """ a HF (halffabricaat)"""

    def __init__(self, name: str, id: str, cost: float, weight: float, waste: float, ingredients: pd.DataFrame) -> None:
        """ initialise an instance of HF"""
        self.name = name
        self.id = id
        self.cost = cost
        self.weight = weight
        self.waste = waste
        self.ingredients = ingredients

## Data preparation ##

### Data loading ###

#### BOM ####


In [5]:
bom_data_raw = pd.read_excel("NAV Recepten Download.xlsx", skiprows=1, header=None, decimal=",")

#### Costs ####

In [6]:
cost_data = pd.read_excel("NAV Recepten Download Extra.xlsx", sheet_name="Prijslijst", skiprows=2)

#### Packaging ####

In [8]:
packaging_data = pd.read_excel("NAV Recepten Download Extra.xlsx", sheet_name="Verpakking", skiprows=2, skipfooter=1)

### Data cleaning ###

#### BOM ####

##### Split data into recipes #####

In [9]:
recipes = []

for i in range(len(bom_data_raw)):
    # a new recipe starts
    if bom_data_raw[4][i] == 'Omschrijving':
        start_idx = i + 1
        recipe_name = bom_data_raw[4][i + 1]
        recipe_id = bom_data_raw[3][i + 1]

        # the recipe ends
        for j in range(i, len(bom_data_raw)):
            if bom_data_raw[3][j] == 'Kostenaandeel voor dit artikel':
                end_idx = j
                recipe_data = bom_data_raw.iloc[(i + 2):j].drop(range(8, 13), axis='columns').reset_index()
                recipe_data = recipe_data.rename(columns={0: "id_nr", 1: "nr", 2: "Niveau", 3: "hf_nr", 4: "Omschrijving", 5: "Aantal (Basis)", 6: "Basiseenheid", 7: "Materiaalkosten"})
                recipe_data = recipe_data.astype({"id_nr": str, "nr": int, "Niveau": int, "hf_nr": str, "Omschrijving": str, "Aantal (Basis)": float, "Basiseenheid": str, "Materiaalkosten": float})
                recipes.append(recipe(recipe_name, recipe_id, recipe_data))
                i += j
                break

##### Split recipes into HFs #####

In [10]:
for recipe in recipes:
    HFs = []
    for i in range(len(recipe.data)):
        if recipe.data["hf_nr"][i].startswith("HF"):
            HF_level = recipe.data["Niveau"][i]
            for j in range(i + 1, len(recipe.data)):
                if not recipe.data["Niveau"][j] <= HF_level:
                    # print(f'a change occurred: {HF_level} {recipe.data["Omschrijving"][i]} to: {recipe.data["Niveau"][j]} {recipe.data["Omschrijving"][j]}')
                    j += 1
                else:
                    break
            HFs.append(HF(name=recipe.data["Omschrijving"][i], id=recipe.data["hf_nr"][i], cost=0.0, weight=0.0, waste=0.0, ingredients=recipe.data.iloc[i + 1 : j + 1 ]))

## Cost calculations ##

In [ ]:
# TODO - create custom cost function

In [11]:
for recipe in recipes:
    recipe_cost = 0
    for index, row1 in recipe.data.iterrows():
        ingredient_nr = row1['hf_nr']
        # filter HFs and packaging
        if not ingredient_nr.startswith("HF") and not (ingredient_nr.startswith("3")):
            for index, row2 in cost_data.iterrows():
                if ingredient_nr == str(int(row2["Rijlabels"])):
                    recipe_cost += float(row2["Gemiddelde van Prijs per Ingredient"]) * float(row1['Aantal (Basis)'])
    recipe.cost = recipe_cost

In [13]:
for HF in HFs:
    HF_cost = 0
    for index, row1 in HF.ingredients.iterrows():
        ingredient_nr = row1['hf_nr']
        # filter HFs and packaging
        if not ingredient_nr.startswith("HF") and not (ingredient_nr.startswith("3")):
            for index, row2 in cost_data.iterrows():
                if ingredient_nr == str(int(row2["Rijlabels"])):
                    HF_cost += float(row2["Gemiddelde van Prijs per Ingredient"]) * float(row1['Aantal (Basis)'])
    HF.cost = HF_cost

In [12]:
for recipe in recipes:
    print(f'{recipe}, {recipe.cost}')

565020 zzHF Kipfilet+champ+r.wijnsaus, 0.009665882352941177
565021 HF Romige tomatenrisotto, 1.605859559261089
565021G HF Romige tomatenrisotto, 1.605859559261089
565021N HF Romige tomatenrisotto, 1.605859559261089


In [14]:
for HF in HFs:
    print(f'{HF.name}, {HF.cost}')

Tomatenrisotto, 0.42597149909420284
Gekookte risotto rijst, 0.10554
Saus voor tomatenrisotto HF, 0.3204314990942029
Geg. cherrytomaat+olie  HF, 0.2256717874396135
Paprika, courg. in olie HF, 0.48883827272727276
HFK Cup pijnboompitten 17g, 0.45556
Gebakken pijnboompitten, 0.45556
Melkzuurbacterie spray, 0.00206
